## Background

### Current dataset (if exists)

PROBA-V is not currently in our RS data list.

### Need for a new data product

#### New dataset description 

The [PROBA-V](https://www.vito-eodata.be/PDF/image/PROBAV-Products_User_Manual.pdf) satellite was launched on 6 May 2013 and flies at an altitude of 820 km in a sun-synchronous orbit with a local overpass time at launch of 10:45 h. The VEGETATION instrument aboard PROBA-V has a Field Of View of 102°, resulting in a swath width of 2295 km. This swath width ensures a daily near-global coverage (90%), whereas the full global coverage is achieved every 2 days. The central camera observes at 100 m nominal resolution, which
covers a swath of about 517 km that ensures global coverage every 5 days. PROBA-V observes in four spectral bands: BLUE (centered at 0.463 µm), RED (0.655 µm), NIR (0.837µm), and SWIR (1.603 µm).

Compared to Landsat-8 (16-day repeat cycle) and Sentinel-2 (10-day repeat cycle), PROBA-V has a better temporal coverage with some sacrifice in spatial resolution (finest-resolution product is at 100m). However, it is worth noting that the data [harmonization activity](https://www.mdpi.com/1424-8220/20/22/6631/htm) of Landsat and Sentinel has reduced the revisit time to 2.3 days, and may be a finer-resolution alternative, especially for the period of June, 2020 onwards when PROBA-V mission [ended operations](https://earth.esa.int/eogateway/news/new-experimental-phase-for-proba-v)


#### Literature research about new data

For the ingestion of PROBA-V, we review the current caveats of existing data and propose the following changes:

* We use the top-of-canopy (TOC) product and select 3 surface reflectance bands (`Red, NIR, SWIR`) and the `NDVI` band as the derived feature. We do not include the `Blue` band, which is more sensitive to atmospheric effect and can be more noisy (e.g. the landsat validation of [non-forest biomes in China](https://www.mdpi.com/2072-4292/14/1/83/htm) and [forest/non-forest biomes in the US](https://www.mdpi.com/2072-4292/11/13/1543)).

* We use the QA flags (`SM` band) in PROBA-V to remove cloud issues. The [cloud and shadow detections](https://www.vito-eodata.be/PDF/image/PROBAV-Products_User_Manual.pdf) in PROBA-V were from a machine-learning approach (cloud) and radiometric/geometric approach. In addition, we try additional filters including SZA and VZA conditions, which is recommended in the same [product manual]((https://www.vito-eodata.be/PDF/image/PROBAV-Products_User_Manual.pdf)) for choosing good quality data.

* The seasonality features are not considered for now, similar to our Landsat case, but will be investigated at a later stage.  

#### Evidences of new data used in biomass studies

This section is to provide evidences in existing literature/projects that the use of proposed data set can improve the biomass estimation. For PROBA-V TOC data set, it is used in vegetation studies, e.g. as the main data source for the Copernicus Global Land Services [(CGLS) Land Cover product](https://land.copernicus.eu/global/products/lc), and the more recent global [forest management data](https://www.nature.com/articles/s41597-022-01332-3).


## Methodology (Processing steps)

This section is mainly to provide code examples of proposed data ingestion. Main steps include

* Working environment setup

* Raw PROBA-V (TOC product without QA)

* PROBA-V TOC with different QA filters


### Setting up environment

In [ ]:
#!pip install geemap

In [26]:
import ee
import geemap.foliumap as geemap
from geemap import colormaps
from functools import partial


In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [19]:
#functions
def gain_offset_sr(image):
    image = image.multiply(0.0005).add(0)
    return image


def gain_offset_vi(image):
    image = image.multiply(0.004).add(-0.08)
    return image


#parameters
param_vis = {
    'min': 0,
    'max': 0.5,
    'bands': ["SWIR", "NIR", "RED"]
}
ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': ['black','red', 'orange', 'yellow', 'green'],
    'bands': ["NDVI",]
}
count_vis = {
    'min': 0,
    'max': 100,
    'bands': ["NIR_count",]
}

### PROBA-V Raw

In [20]:
# time range
year = 2019
start_date = ee.Date.fromYMD(year, 1, 1)
end_date = ee.Date.fromYMD(year, 12, 31)


# image collection
bands = ["RED", "NIR", "SWIR", "NDVI"]
filtered = (
    ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M')
        .filterDate(start_date, end_date)
        .select(bands)
)

# feature extraction
features = filtered.median()

# normalization
sr = gain_offset_sr(features.select(bands[:3]))
vi = gain_offset_vi(features.select(bands[3:4]))
out_features = sr.addBands(vi)   # Main Output bands
print(out_features.getInfo())

count = filtered.reduce(ee.Reducer.count()) # for visual check
print(count.bandNames().getInfo())

# visualization
Map = geemap.Map(center=(-5, -50), zoom=6)
Map.addLayer(count, count_vis, 'count')
Map.addLayer(out_features, ndvi_vis, 'ndvi')
Map.addLayer(out_features, param_vis, 'PROBA-V')

Map

{'type': 'Image', 'bands': [{'id': 'RED', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'NIR', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SWIR', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -0.08, 'max': 0.9400000000000001}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
['RED_count', 'NIR_count', 'SWIR_count', 'NDVI_count']


### PROBA-V Filtered

#### Cloud/Ice masking, Annual median composites

In [21]:
# masking
def maskClouds(image):
    flag1 = ee.Number(127);  # binary: 01111111

    img = image.updateMask(
      image.select(["SM"]).bitwiseAnd(flag1).eq(120)   # binary: 01111000
    )
    return img


# time range
year = 2019
start_date = ee.Date.fromYMD(year, 1, 1)
end_date = ee.Date.fromYMD(year, 12, 31)


# image collection
bands = ["RED", "NIR", "SWIR", "NDVI"]
filtered = (
    ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M')
        .filterDate(start_date, end_date)
        .map(maskClouds)
        .select(bands)
)

# feature extraction
features = filtered.median()

# normalization
sr = gain_offset_sr(features.select(bands[:3]))
vi = gain_offset_vi(features.select(bands[3:4]))
out_features = sr.addBands(vi)   # Main Output bands
print(out_features.getInfo())

count = filtered.reduce(ee.Reducer.count()) # for visual check
print(count.bandNames().getInfo())

# visualization
Map = geemap.Map(center=(-5, -50), zoom=6)
Map.addLayer(count, count_vis, 'count')
Map.addLayer(out_features, ndvi_vis, 'ndvi')
Map.addLayer(out_features, param_vis, 'PROBA-V')

Map

{'type': 'Image', 'bands': [{'id': 'RED', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'NIR', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SWIR', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -0.08, 'max': 0.9400000000000001}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
['RED_count', 'NIR_count', 'SWIR_count', 'NDVI_count']


#### More QA masking, Annual median composites

In [22]:
# masking
def maskClouds(image):
    flag1 = ee.Number(127);  # binary: 01111111

    img = image.updateMask(
      image.select(["SM"]).bitwiseAnd(flag1).eq(120)   # binary: 01111000
      .And(image.select(["NIR"]).gte(0))
      .And(image.select(["NDVI"]).lt(255))
      .And(image.select(["SZA"]).lt(120))   # scaling x2
      .And(image.select(["VNIRVZA"]).lt(120))   # scaling x2
    )
    return img


# time range
year = 2019
start_date = ee.Date.fromYMD(year, 1, 1)
end_date = ee.Date.fromYMD(year, 12, 31)


# image collection
bands = ["RED", "NIR", "SWIR", "NDVI"]
filtered = (
    ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M')
        .filterDate(start_date, end_date)
        .map(maskClouds)
        .select(bands)
)

# feature extraction
features = filtered.median()

# normalization
sr = gain_offset_sr(features.select(bands[:3]))
vi = gain_offset_vi(features.select(bands[3:4]))
out_features = sr.addBands(vi)   # Main Output bands
print(out_features.getInfo())

count = filtered.reduce(ee.Reducer.count()) # for visual check
print(count.bandNames().getInfo())

# visualization
Map = geemap.Map(center=(-5, -50), zoom=6)
Map.addLayer(count, count_vis, 'count')
Map.addLayer(out_features, ndvi_vis, 'ndvi')
Map.addLayer(out_features, param_vis, 'PROBA-V')

Map

{'type': 'Image', 'bands': [{'id': 'RED', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'NIR', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SWIR', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -0.08, 'max': 0.9400000000000001}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
['RED_count', 'NIR_count', 'SWIR_count', 'NDVI_count']


#### More QA masking, max ndvi composites

In [23]:
# masking
def maskClouds(image):
    flag1 = ee.Number(127);  # binary: 01111111

    img = image.updateMask(
      image.select(["SM"]).bitwiseAnd(flag1).eq(120)   # binary: 01111000
      .And(image.select(["NIR"]).gte(0))
      .And(image.select(["NDVI"]).lt(255))
      .And(image.select(["SZA"]).lt(120))   # scaling x2
      .And(image.select(["VNIRVZA"]).lt(120))   # scaling x2
    )
    return img


# time range
year = 2019
start_date = ee.Date.fromYMD(year, 1, 1)
end_date = ee.Date.fromYMD(year, 12, 31)


# image collection
bands = ["RED", "NIR", "SWIR", "NDVI"]
filtered = (
    ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M')
        .filterDate(start_date, end_date)
        .map(maskClouds)
        .select(bands)
)

# feature extraction
features = filtered.qualityMosaic('NDVI')

# normalization
sr = gain_offset_sr(features.select(bands[:3]))
vi = gain_offset_vi(features.select(bands[3:4]))
out_features = sr.addBands(vi)   # Main Output bands
print(out_features.getInfo())

count = filtered.reduce(ee.Reducer.count()) # for visual check
print(count.bandNames().getInfo())

# visualization
Map = geemap.Map(center=(-5, -50), zoom=6)
Map.addLayer(count, count_vis, 'count')
Map.addLayer(out_features, ndvi_vis, 'ndvi')
Map.addLayer(out_features, param_vis, 'PROBA-V')

Map

{'type': 'Image', 'bands': [{'id': 'RED', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'NIR', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SWIR', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -0.08, 'max': 0.9400000000000001}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
['RED_count', 'NIR_count', 'SWIR_count', 'NDVI_count']


####More QA Masking, ndvi thresholding, interval median

In [27]:
# masking
def maskClouds(image):
    flag1 = ee.Number(127);  # binary: 01111111

    img = image.updateMask(
      image.select(["SM"]).bitwiseAnd(flag1).eq(120)   # binary: 01111000
      .And(image.select(["NIR"]).gte(0))
      .And(image.select(["NDVI"]).lt(255))
      .And(image.select(["SZA"]).lt(120))   # scaling x2
      .And(image.select(["VNIRVZA"]).lt(120))   # scaling x2
    )
    return img


def maskNDVI(image, med_ndvi):
    img = image.updateMask(
        image.select(['NDVI']).gte(med_ndvi)
    )
    return img


# time range
year = 2019
start_date = ee.Date.fromYMD(year, 1, 1)
end_date = ee.Date.fromYMD(year, 12, 31)


# image collection
bands = ["RED", "NIR", "SWIR", "NDVI"]
filtered = (
    ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M')
        .filterDate(start_date, end_date)
        .map(maskClouds)
        .select(bands)
)

# feature extraction
med_ndvi = filtered.median().select(['NDVI'])
maskNDVI_med = partial(maskNDVI, med_ndvi=med_ndvi)
features = filtered.map(maskNDVI_med).median()


# normalization
sr = gain_offset_sr(features.select(bands[:3]))
vi = gain_offset_vi(features.select(bands[3:4]))
out_features = sr.addBands(vi)   # Main Output bands
print(out_features.getInfo())

count = filtered.reduce(ee.Reducer.count()) # for visual check
print(count.bandNames().getInfo())

# visualization
Map = geemap.Map(center=(-5, -50), zoom=6)
Map.addLayer(count, count_vis, 'count')
Map.addLayer(out_features, ndvi_vis, 'ndvi')
Map.addLayer(out_features, param_vis, 'PROBA-V')

Map

{'type': 'Image', 'bands': [{'id': 'RED', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'NIR', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SWIR', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -16.384, 'max': 16.3835}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -0.08, 'max': 0.9400000000000001}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
['RED_count', 'NIR_count', 'SWIR_count', 'NDVI_count']
